In [ ]:
q1_admin = vaccine_eda['Administered'].quantile(0.25)
q3_admin = vaccine_eda['Administered'].quantile(0.75)

# Calculating the IQR close by subtracting Q1 close
#   from Q3 adminstered
iqr_admin = q3_admin - q1_admin

# We calculate the lower and upper ranges of the IQR
#   by subtracting and adding 1.5 from the IQR then
#   multiplying it.

lower_admin = q1_admin - 1.5 * iqr_admin
upper_admin = q1_admin + 1.5 * iqr_admin

# Do the same for Distributed
q1_dist = vaccine_eda['Distributed'].quantile(0.25)
q3_dist = vaccine_eda['Distributed'].quantile(0.75)
iqr_dist = q3_dist - q1_dist

lower_dist = q1_dist - 1.5 * iqr_dist
upper_dist = q1_dist + 1.5 * iqr_dist
Now we can plot the IQR with a boxplot.
# Creating subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Box plot for closing
axes[0].boxplot(vaccine_eda['Administered'], vert=True)
axes[0].axhline(lower_admin, color='red', linestyle='--', label='Lower Bound')
axes[0].axhline(upper_admin, color='green', linestyle='--', label='Upper Bound')
axes[0].set_title('Box Plot of Administered Vaccines')
axes[0].set_ylabel('Administrations')
axes[0].legend()

# Box plot for volume
axes[1].boxplot(vaccine_eda['Distributed'], vert=True)
axes[1].axhline(lower_dist, color='red', linestyle='--', label='Lower Bound')
axes[1].axhline(upper_dist, color='green', linestyle='--', label='Upper Bound')
axes[1].set_title('Box Plot of Distributed Vaccines')
axes[1].set_ylabel('Distributions')
axes[1].legend()


plt.tight_layout()
plt.show()


<b>NOTE TO SELF:</b>
ANOVA is only for categorical variables.

The below was incorrectly implemented and is an archived excerpt from the Term Paper.

#### One-Way ANOVA

In order to perform an ANOVA the data will need to be converted into long format with the Stages of comletion as the x and Supply as the y.

A one-way ANOVA tests whether the means of a contiuous variable differe significantly accross two or more independent groups. This differs from a two-way (or factorial) ANOVA by testing the effects of independent categorical variables on a continous dependent variable. This is the correct ANOVA because `Supply` is the continous variable we are comparing to the stages which are the independent groups.

In [ ]:
# Build stage column headers
stages = ['Recip_Administered', 'Administered_Dose1_Recip', 'Series_Complete_Yes', 'Additional_Doses', 'Second_Booster', 'Administered_Bivalent']

# Prepare data for anova by melting 
anova = vaccine_eda.melt(id_vars=['Supply'], value_vars=stages, # Make supply the values
                                  var_name='Stage', value_name='StageValue') # Stages are the fields

The goal is to determine if at least one group mean differs from the other.

`spicy` has a module `f_oneway` which allows us to perform one way ANOVA.

In [ ]:
# Create a list of groups where each group contains the 'Supply' values
#   for a specific 'Stage' in the dataset. This is used to prepare data
#   for ANOVA, grouping by each unique stage.
groups = [anova.loc[anova['Stage'] == stage, 'Supply'] for stage in stages]

# Perform one-way ANOVA test on the grouped 'Supply' data to determine
# if there are statistically significant differences in mean supply across stages.
f_stat, p_val = f_oneway(*groups)


In [ ]:
print("F-Stat:", f"{f_stat:.50f}")
print(f"P-value:", p_val)


F-Stat: 0.00000000000000000000000000000392100795322662234280
P-value: 1.0


The F-Stat for the hypothesis of supply being statistically significant to the different stages of vaccine completion is extremely close to zero.

This means there's almost no statistical difference between the Supply and patient therapy.

A p-value of 1.0 means there is no statistical evidence to reject the null hypothesis. In other words, there is no statistical evidence that Supply differs access Stages of vaccine completion. According to this analysis, Supply does not appear to have any measurable effect on vaccine completion outcomes.

It is no longer meaningful to compare the Supply with completed stages. Instead a new hypothsis is required.

The type of manufacturer may be more likely to have an affect on patient therapy.

Our new hypothesis is:
#### <i>"The type of vaccine manufacturer has an affect on the stages of patient outcomes."</i>

This is the new statement we are trying to prove.

#### Two-Way ANOVA

Using Supply as a predictor in the previous hypotesis wa appropriate because we had one